# Web-scrapping

Иногда нужно получить данные из интернета. Это можно сделать через API или напрямую с сайта. Получение информации напрямую с сайта - это web scrapping. 

Инструменты для скраппинга:

1. Developer Tools в браузере;
2. Requests;
3. Beautiful Soup;
4. Requests-html;
5. Selenium.

В примере ниже видно в заголовках запроса, что сайт знает о нас, как о приложении, написанном на Python. Чтобы это обхитрить, можно использовать fake_headers:

In [2]:
# pip install requests
# pip install beautifulsoup4
# pip install lxml
# pip install fake_headers

import re
import requests
from fake_headers import Headers

response = requests.get("https://2ip.ru/")
print(response.request.headers)

headers = Headers(browser='firefox', os='linux').generate()
response = requests.get("https://2ip.ru/", headers=headers)
print(response.request.headers)

addr_template = re.search(r'<div class="ip" id="d_clip_button".*?</div>', response.text, re.DOTALL).group()
addr = re.search(r'<span>.*?</span>', addr_template).group()
ip_address = addr.strip('</span>')

print(ip_address)

{'User-Agent': 'python-requests/2.32.3', 'Accept-Encoding': 'gzip, deflate', 'Accept': '*/*', 'Connection': 'keep-alive'}
{'User-Agent': 'Mozilla/5.0 (X11; Linux i686; rv:67.0.2) Gecko/20100101 Firefox/67.0.2', 'Accept-Encoding': 'gzip, deflate', 'Accept': '*/*', 'Connection': 'keep-alive'}
37.18.93.10


## Про BeautifulSoup

У объекта BeautifulSoup есть аттрибут text. Этот аттрибут хранит в себе текст узла без тэгов

Важно, что элемент BeautifulSoup также содержит и аттрибуты тега. Получить их можно по ключу ['<имя аттрибута>']

Методы:

* find (<назв. тэга>, <через именованные переменные или attrs уточняем остальное>) - ищет узел с указанным тэгом и если уточнено, с таким то классом, таким то селектором и тд. Возвращает элемент soup attrs - это словарик с аттрибутами. Пример: attrs={'class':'id'}
* find_all (<назв. тэга>, <через именованные переменные или attrs уточняем остальное>) - ищет все узлы с указанным тэгом и если уточнено, с таким то классом, таким то селектором и тд. Возвращает список attrs - это словарик с аттрибутами. Пример: attrs={'class':'id'}
* select_one (<css селектор>) - возвращает первый элемент html с соответствующим css селектором
* select (<css селектор>) - возвращает все элементы html с соответствующим css селектором.

In [3]:
import bs4

soup = bs4.BeautifulSoup(response.text, features='lxml')
div_tag = soup.find('div', attrs={'class':'ip'})
span_tag = div_tag.find('span')
print(span_tag.text)

37.18.93.10


In [1]:
import bs4
import json
import requests
from pprint import pprint

response = requests.get("https://dtf.ru/games")
soup = bs4.BeautifulSoup(response.text, features='lxml')

article_list = soup.select_one('div.content-list')
articles = soup.select('div.content--short')

parsed_data = []

for article in articles:
    link = 'https://dtf.ru' + article.select_one('a.content__link')['href']
    article_response = requests.get(link)
    article_soup = bs4.BeautifulSoup(article_response.text, features='lxml')

    title = article_soup.select_one('h1.content-title').text.strip()
    content = article_soup.select_one('article.content__blocks').text.replace('\"', '\'')
    date = article_soup.select_one('div.content-header-details__inner').\
        select_one('a.noshrink').\
        select_one('time')['title']
    
    parsed_data.append({
        'link': link,
        'title': title,
        'content': content,
        'date': date
    })
    

with open('data.json', 'w', encoding='utf-8') as file:
    json.dump(parsed_data, file, ensure_ascii=False, indent=4)


## Про Selenium

Это библиотека, которая позволяет работать с драйвером браузера. Для корректной и удобной работы с этой библиотекой, необходим менеджер драйверов: webdriver-manager

При получении странички есть одна проблема. Не все элементы прогружаются моментально, и есть шанс, что в момент запроса у сайта некоторого элемента, сайт может пока не знать о нем (он еще не прогрузится к тому моменту). Поэтому можно воспользоваться функцией wait_element() написанной автором. Она дожидается элемента и возвращает его.

* find_element - найти первый попавшийся такой элемент;
* find_elements - найти все такие элементы.

В возвращаемом элементе мы можем продолжать искать, что захотим

С помощью Selenium можно устроить интерактив с веб-страницей:

* click() - кликает;
* send_keys() - посылает либо строку, либо клавишу. (клавиши в Keys импорт из selenium.webdriver)

In [2]:
# почему-то код не работает. См. прикрепленный файл selenium_test.py

import json
import time
from selenium.common import TimeoutException
from selenium import webdriver
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By

options = Options()
# options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

def wait_element(browser, delay=3, by=By.TAG_NAME, value=None):
    try: 
        return WebDriverWait(browser, delay).until(
            expected_conditions.presence_of_element_located((by, value))
        )
    except TimeoutException:
        return None

driver.get("https://dtf.ru/games")
time.sleep(1)

articles = driver.find_elements(by=By.CSS_SELECTOR, value='div.content--short')
links = []
for article in articles:
    link = wait_element(browser=article, by=By.CSS_SELECTOR, value='a.content__link').get_attribute('href')
    links.append(link)

data = []
for link in links:
    driver.get(link)
    title = wait_element(browser=driver, by=By.CSS_SELECTOR, value='h1.content-title').text
    content = wait_element(browser=driver, by=By.CSS_SELECTOR, value='article.content__blocks').text.replace('\"', '\'')
    date = wait_element(browser=driver, by=By.TAG_NAME, value='time').get_attribute('title')
    data.append({
        'link': link,
        'title': title,
        'content': content,
        'date': date
    })
    
with open('data_selenium.json', 'w', encoding='utf-8') as file:
    json.dump(data, file, ensure_ascii=False, indent=4)

driver.close()

KeyboardInterrupt: 

In [ ]:
import json
import time
from selenium.common import TimeoutException
from selenium import webdriver
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By

options = Options()
# options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--fullscreen')
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

def wait_element(browser, delay=3, by=By.TAG_NAME, value=None):
    try: 
        return WebDriverWait(browser, delay).until(
            expected_conditions.presence_of_element_located((by, value))
        )
    except TimeoutException:
        return None

driver.maximize_window()
driver.get("https://dtf.ru/games")
time.sleep(1)

search_button = wait_element(driver, by=By.CSS_SELECTOR, value='button.search__button')
search_button.click()  # нажатие на кнопку
time.sleep(1)
search_input = wait_element(driver, by=By.CSS_SELECTOR, value='input.text-input')
search_input.send_keys('папич')
time.sleep(1)

driver.close()
